In [1]:

library(terra)
library(sf)
library(tidyterra)
library(MultiscaleDTM)
library(randomForest)
library(caret)
library(vip)
library(rstac)
library(whitebox)
library(tmap)

s_obj <- stac("https://planetarycomputer.microsoft.com/api/stac/v1")

terra 1.7.71

Linking to GEOS 3.10.2, GDAL 3.4.3, PROJ 8.2.0; sf_use_s2() is TRUE


Attaching package: ‘tidyterra’


The following object is masked from ‘package:stats’:

    filter


Warning message in rgl.init(initValue, onlyNULL):
“RGL: unable to open X11 display”
Warning message:
“'rgl.init' failed, running with 'rgl.useNULL = TRUE'.”
randomForest 4.7-1.1

Type rfNews() to see new features/changes/bug fixes.

Loading required package: ggplot2


Attaching package: ‘ggplot2’


The following object is masked from ‘package:randomForest’:

    margin


Loading required package: lattice


Attaching package: ‘vip’


The following object is masked from ‘package:utils’:

    vi



Attaching package: ‘tmap’


The following object is masked from ‘package:datasets’:

    rivers




In [2]:
wa_dem_prj <- rast("data/wa_dem_diss_prj3740.tif")
wa_shp <- vect("data/wa_shp_diss.gpkg")

In [3]:
terraOptions(memmax=0.01, memmin=0.01)

In [4]:
mem_info(wa_dem_prj)


------------------------
Memory (GB) 
------------------------
check threshold : 0.01 (memmin)
available       : 0.01 (memmax)
allowed (60%)   : 0.01
needed (n=1)    : 3.21
------------------------
proc in memory  : FALSE
nr chunks       : 893
------------------------


In [5]:
wa_dem_prj

class       : SpatRaster 
dimensions  : 16959, 25378, 1  (nrow, ncol, nlyr)
resolution  : 23.64304, 23.64304  (x, y)
extent      : 371131.6, 971144.8, 5043580, 5444542  (xmin, xmax, ymin, ymax)
coord. ref. : NAD83(HARN) / UTM zone 10N (EPSG:3740) 
source      : wa_dem_diss_prj3740.tif 
name        : NASADEM_HGT_n49w125 
min value   :           -35.65501 
max value   :          4373.03906 

In [4]:
wa_shp

 class       : SpatVector 
 geometry    : polygons 
 dimensions  : 1, 0  (geometries, attributes)
 extent      : 371124.2, 971138, 5043576, 5444546  (xmin, xmax, ymin, ymax)
 source      : wa_shp_diss.gpkg
 coord. ref. : NAD83(HARN) / UTM zone 10N (EPSG:3740) 

In [14]:
# wbt_slope(
#     dem = "data/stack_data/wa_dem_diss_prj3740.tif",
#     output = "data/stack_data/wa_slope.tif"
# )

[1] "slope - Elapsed Time (excluding I/O): 7.213s"

In [21]:
wbt_multiscale_roughness(
    dem = "data/stack_data/wa_dem_diss_prj3740.tif",
    out_mag = "data/stack_data/wa_roughmag.tif",
    out_scale = "data/stack_data/wa_roughscale.tif",
    min_scale = 1,
    max_scale = 20,
    step = 5
)

Warning message in system(args2, intern = TRUE):
“running command '/home/ajs0428/R/x86_64-pc-linux-gnu-library/4.3/whitebox/WBT/whitebox_tools --run=multiscale_roughness  --dem=data/stack_data/wa_dem_diss_prj3740.tif --out_mag=data/stack_data/wa_roughmag.tif --out_scale=data/stack_data/wa_roughscale.tif --max_scale=20 --min_scale=1 --step=5 -v' had status 137”


[1] "multiscale_roughness - Reading data..."

In [16]:
wbt_plan_curvature(
    dem = "data/stack_data/wa_dem_diss_prj3740.tif",
    output = "data/stack_data/wa_planc.tif"
)

[1] "plan_curvature - Elapsed Time (excluding I/O): 6.561s"

In [17]:
wbt_profile_curvature(
    dem = "data/stack_data/wa_dem_diss_prj3740.tif",
    output = "data/stack_data/wa_profc.tif"
)

[1] "profile_curvature - Elapsed Time (excluding I/O): 7.51s"

In [18]:
wbt_d_inf_pointer(
    dem = "data/stack_data/wa_dem_diss_prj3740.tif",
    output = "data/stack_data/wa_dinfpoint.tif"
)

[1] "d_inf_pointer - **********************************************************************************"

In [19]:
wbt_d_inf_flow_accumulation(
    i = "data/stack_data/wa_dinfpoint.tif",
    output = "data/stack_data/wa_dinfsca.tif"
)

[1] "d_inf_flow_accumulation - **********************************************************************************"

In [7]:
wbt_wetness_index(
    sca = "data/stack_data/wa_dinfsca.tif",
    slope = "data/stack_data/wa_slope.tif",
    output = "data/stack_data/wa_twi.tif"
)

[1] "wetness_index - Elapsed Time (excluding I/O): 5.331s"

In [9]:
wbt_ruggedness_index(
    dem = "data/stack_data/wa_dem_diss_prj3740.tif",
    output = "data/stack_data/wa_rugged.tif"
)

[1] "ruggedness_index - Elapsed Time (excluding I/O): 5.346s"

In [10]:
?terra::focal

focal {terra},R Documentation
x,SpatRaster
w,"window. The window can be defined as one (for a square) or two numbers (row, col); or with an odd-sized weights matrix. See Details."
fun,"function that takes multiple numbers, and returns a numeric vector (one or multiple numbers). For example mean, modal, min or max"
...,additional arguments passed to fun such as na.rm
na.policy,"character. Can be used to determine the cells of x for which focal values should be computed. Must be one of ""all"" (compute for all cells), ""only"" (only for cells that are NA) or ""omit"" (skip cells that are NA). Note that the value of this argument does not affect which cells around each focal cell are included in the computations (use na.rm=TRUE to ignore cells that are NA for that)"
fillvalue,numeric. The value of the cells in the virtual rows and columns outside of the raster
expand,"logical. If TRUE The value of the cells in the virtual rows and columns outside of the raster are set to be the same as the value on the border. Only available for ""build-in"" funs such as mean, sum, min and max"
silent,logical. If TRUE error messages are printed that may occur when trying fun to determine the length of the returned value. This can be useful in debugging a fun that does not work
filename,character. Output filename
overwrite,"logical. If TRUE, filename is overwritten"


In [ ]:
# wa_curv3 <- MultiscaleDTM::Qfit(wa_dem_prj, w = c(3,3), metrics = c("profc", "planc", "meanc"),
#                                filename = "data/wa_curv3.tif")

In [ ]:
# system.time(MultiscaleDTM::Qfit(wa_dem_prj, w = c(27,27), metrics = c("profc", "planc", "meanc"),
#                                filename = "data/wa_curv27.tif"))

In [ ]:
# wa_curv81 <- MultiscaleDTM::Qfit(wa_dem_prj, w = c(81,81), metrics = c("profc", "planc", "meanc"),
#                                filename = "data/wa_curv27.tif")

In [6]:
?wbt_wetness_index

wbt_wetness_index {whitebox},R Documentation
sca,Input raster specific contributing area (SCA) file.
slope,Input raster slope file (in degrees).
output,Output raster file.
wd,Changes the working directory.
verbose_mode,"Sets verbose mode. If verbose mode is False, tools will not print output messages."
